# 리뷰 특성 벡터화 후 클러스터링
## Import

In [1]:
import pandas as pd
from pandas import DataFrame

import numpy as np

import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

from sklearn.decomposition import PCA

from tqdm import tqdm

from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from konlpy.tag import Okt

from sklearn.cluster import DBSCAN

### Data Load

In [2]:
Raw_Data = pd.read_csv("./Original_Data_UTF8.CSV")

Data_Set = pd.DataFrame(Raw_Data["Review"])

Data_Set.head()

,Review
0,제발 리뷰 솔직하게 씁시다~ 닭인지 병아리인지 모르겠고 본점이라면서 짱깨 집도 같이...
1,맛은 있는데 치킨이 진짜 너무 짜게 식어서 옴...... 먹는데 이거 항의할까 말까...
2,불고기버거 단품 2개와 한우버거 세트를 주문했는데 콜라와 포테이토가 빠진 단품 3개...
3,말도 없이 배달 엄청 늦어짐
4,순두부찌개는 너무 짠맛만 나고.. 물 넣어서 전자레인지 돌려먹었어요 그냥 딱 쉰 김...


In [3]:
Raw_Data.head()

,Review,Sentiment,Rate,Rate_Label
0,제발 리뷰 솔직하게 씁시다~ 닭인지 병아리인지 모르겠고 본점이라면서 짱깨 집도 같이...,-1,1,-2
1,맛은 있는데 치킨이 진짜 너무 짜게 식어서 옴...... 먹는데 이거 항의할까 말까...,-1,2,-1
2,불고기버거 단품 2개와 한우버거 세트를 주문했는데 콜라와 포테이토가 빠진 단품 3개...,-1,1,-2
3,말도 없이 배달 엄청 늦어짐,-1,1,-2
4,순두부찌개는 너무 짠맛만 나고.. 물 넣어서 전자레인지 돌려먹었어요 그냥 딱 쉰 김...,-1,2,-1


In [4]:
Data_Set['length'] = ''

RawLength = []

for index in tqdm(Data_Set['Review']):
    # Data_Set['length'] = len(str(index))
    # Current_Length = len(str(index))
    # Data_Set.length[index] = Current_Length
    # print(len(str(index)))
    
    RawLength.append(len(str(index)))

Data_Set['length'] = RawLength

Data_Set.head()

100%|███████████████████████████████████████████████████████████████████████| 10658/10658 [00:00<00:00, 1332425.99it/s]


,Review,length
0,제발 리뷰 솔직하게 씁시다~ 닭인지 병아리인지 모르겠고 본점이라면서 짱깨 집도 같이...,53
1,맛은 있는데 치킨이 진짜 너무 짜게 식어서 옴...... 먹는데 이거 항의할까 말까...,163
2,불고기버거 단품 2개와 한우버거 세트를 주문했는데 콜라와 포테이토가 빠진 단품 3개...,300
3,말도 없이 배달 엄청 늦어짐,15
4,순두부찌개는 너무 짠맛만 나고.. 물 넣어서 전자레인지 돌려먹었어요 그냥 딱 쉰 김...,212


### 리뷰 String 처리

In [5]:
Review_Text = Data_Set['Review']

Review_Text.head()

# Review_Text_DF =  pd.DataFrame(Review_Text_Series)

# dir(Review_Text_Series)

0    제발 리뷰 솔직하게 씁시다~ 닭인지 병아리인지 모르겠고 본점이라면서 짱깨 집도 같이...
1    맛은 있는데 치킨이 진짜 너무 짜게 식어서 옴...... 먹는데 이거 항의할까 말까...
2    불고기버거 단품 2개와 한우버거 세트를 주문했는데 콜라와 포테이토가 빠진 단품 3개...
3                                      말도 없이 배달 엄청 늦어짐
4    순두부찌개는 너무 짠맛만 나고.. 물 넣어서 전자레인지 돌려먹었어요 그냥 딱 쉰 김...
Name: Review, dtype: object

In [6]:
okt = Okt()

tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=10, max_df=0.9, tokenizer=okt.morphs, token_pattern=None)

tfidf.fit(Review_Text.values.astype('U'))

Vector_Result = tfidf.transform(Review_Text.values.astype('U'))

Vector_Result_DF = pd.DataFrame(Vector_Result.toarray())

Vector_Result_DF.head()

# print(Review_Vector_Final)

# print(Data_Set['Review'].values.astype('U'))

,0,1,2,3,4,5,6,7,8,9,...,3495,3496,3497,3498,3499,3500,3501,3502,3503,3504
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.083776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
Length_Data = Data_Set['length']

Vector_Result_DF['length'] = Length_Data

Sentimental_Data = Raw_Data['Sentiment']

Vector_Result_DF['Sentiment'] = Sentimental_Data

Vector_Result_DF.head()

,0,1,2,3,4,5,6,7,8,9,...,3497,3498,3499,3500,3501,3502,3503,3504,length,Sentiment
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53,-1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,163,-1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300,-1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,-1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,212,-1


In [8]:
pca = PCA(n_components=2)
Vector_Result_DF_dimensionality_reduction = pca.fit_transform(Vector_Result_DF)

Vector_Result_DF_dimensionality_reduction = pd.DataFrame(Vector_Result_DF_dimensionality_reduction)

Vector_Result_DF_dimensionality_reduction.head()

,0,1
0,7.689536,-1.405195
1,117.688027,-0.815256
2,254.685936,-0.071039
3,-30.309919,-1.609762
4,166.687286,-0.549040


In [9]:
Cluster_Model = DBSCAN(eps=0.5, min_samples=4)

DBSCAN_Result = Cluster_Model.fit(Vector_Result_DF_dimensionality_reduction)

Data_Set['Cluster_Labels'] = DBSCAN_Result.labels_

Data_Set.head()

,Review,length,Cluster_Labels
0,제발 리뷰 솔직하게 씁시다~ 닭인지 병아리인지 모르겠고 본점이라면서 짱깨 집도 같이...,53,0
1,맛은 있는데 치킨이 진짜 너무 짜게 식어서 옴...... 먹는데 이거 항의할까 말까...,163,-1
2,불고기버거 단품 2개와 한우버거 세트를 주문했는데 콜라와 포테이토가 빠진 단품 3개...,300,-1
3,말도 없이 배달 엄청 늦어짐,15,1
4,순두부찌개는 너무 짠맛만 나고.. 물 넣어서 전자레인지 돌려먹었어요 그냥 딱 쉰 김...,212,-1


In [12]:
Data_Set.to_csv('./result.csv', sep=',', na_rep='NaN', encoding = 'cp949')

### 시각화